In [18]:
import requests
from bs4 import BeautifulSoup as bs

def getPageContent(url):
    res = requests.get(url)
    soup = bs(res.text, 'html.parser')
    return soup.select('div.articulum.trans')[0].text.split('googletag')[0].strip()

In [19]:
import requests
from bs4 import BeautifulSoup as bs
import re
import time

localtime = time.strftime("%H/%M")
domain = 'http://www.appledaily.com.tw/'
stopFlag = False

def getPageNews(page=1):
    res = requests.get('http://www.appledaily.com.tw/realtimenews/section/new/{}'.format(page))
    soup = bs(res.text, 'html.parser')

#     print(soup.select('h1.dddd')[0].text)
#     if today==soup.select('h1.dddd')[0].text break

    today = "".join(soup.select('h1.dddd')[0].text.split())
    news = []
    time = today+' 23:59:59'
#     print(len(soup.select('li.rtddt')))
    for li in soup.select('li.rtddt'):
#     print("===================")
        h1 = li.select('h1')[0].text
        m = re.match('(.+)\((\d+)\)', h1)
        if(time < '{} {}'.format(today, li.select('time')[0].text)):
            global stopFlag 
            stopFlag = True
            break
        time  = '{} {}'.format(today, li.select('time')[0].text)
        category = li.select('h2')[0].text
        link = domain + li.select('a')[0]['href']
        if m:
            title =  m.group(1)
            view_cnt = int(m.group(2))
#       print(title, view_cnt, time, category, link)
            news.append({'title':title, 'view_cnt':view_cnt, 'time': time, 'category': category, 'link': link, 'summary': getPageContent(link)}) 
    return news

In [20]:
newsToday=[]
page=1
while True:
    print(page, end=' ')
    newsToday.extend(getPageNews(page));
    if(stopFlag):
        break
    page += 1;    
# print(newsToday)

1 2 3 4 5 6 7 8 9 

In [21]:
# 各類別，取head()，並依view_cnt排序，類似各類別的 top N 新聞
import pandas as pd 
df = pd.DataFrame(newsToday)

newslist = []
for cat in sorted(df['category'].unique(), reverse=True):
    news= df[df.category==cat].sort_values(by=['view_cnt'],ascending=False).head(3)
    newslist.append(news)
result = pd.concat(newslist)
result.pop("link")
result.loc[:,['time','category','title','view_cnt']]

,time,category,title,view_cnt
45,2016/05/12 09:44,體育,【更新】陳偉殷投6.1局送12K失2分好...,70339
186,2016/05/12 00:31,體育,滑球伸卡到變速球 37歲的陽建福升級,12302
41,2016/05/12 09:47,體育,【更新】皇家7局7：3領先 建仔尚未登板,10220
9,2016/05/12 10:14,財經,【更新】三菱油耗造假陷困頓 三菱證實可能...,6388
68,2016/05/12 09:00,財經,這兩人爭中鋼董座 背後是小英跟花媽,4476
207,2016/05/12 00:11,財經,【鴻夏戀】夏普新社長、新董事、2015年...,3309
209,2016/05/12 00:10,論壇,鄭捷的病態人格怎麼養成,13733
219,2016/05/12 00:04,論壇,鳥來伯：男人外遇是誰的錯,5886
205,2016/05/12 00:12,論壇,三聲槍響後 留下了什麼,5689
124,2016/05/12 06:13,社會,鄭捷簽下求生狀 最後努力晚13分鐘,208188


In [4]:
# -*-coding: utf-8 -*-
import sqlite3
db=sqlite3.connect('news.sqlite')
cur=db.cursor()
cur.execute('''DROP TABLE IF EXISTS news_entry''')
cur.execute('''CREATE TABLE IF NOT EXISTS news_entry
     (id INTEGER PRIMARY KEY,
      category TEXT,
      view_cnt INTEGER,
      time TEXT,
      title TEXT,
      summary TEXT,
      link TEXT)''')
cur.close()

In [ ]:
import sqlite3
db=sqlite3.connect('news.sqlite')
cur=db.cursor()
insert_stmt = 'insert into news_entry({}) values({})'
for idx,news in enumerate(newsToday):
    print(idx, end=' ')
    columns = ','.join(news.keys())
    vals = ','.join(len(news.keys()) * '?')
    sql = insert_stmt.format(columns, vals)
#     print(sql)
#     print(news.values())
    cur.execute(sql, list(news.values())) # 要list()轉型
    db.commit() # 要commit
cur.close()
db.close()

# Dataframe to DB

In [12]:
import pandas as pd 
df = pd.DataFrame(newsToday)

import sqlite3 as lite
with lite.connect('news.sqlite') as db:
    df.to_sql(name='news_entry', index=False, con=db, if_exists='replace')
db.close()

In [10]:
# Drop table 
import sqlite3
db=sqlite3.connect('news.sqlite')
cur=db.cursor()
sql = 'drop table news_entry3'
try:
    cur.execute(sql)
except:
    print("Error Occur!")
    traceback.print_exc()
finally:    
    db.commit() # 要commit
    cur.close()
    db.close()